In [ ]:
#@title **STEP1 :** Upload Master Attendance File in **.xlsx** format
!pip install xlsxwriter &> /dev/null
!pip install openpyxl &> /dev/null
!pip install xlrd &> /dev/null

from google.colab import files
from openpyxl import *
from termcolor import colored
import xlrd
import xlsxwriter
import pandas as pd

def getColumnName(n):
    result = ""
    while n > 0:
        index = (n - 1) % 26
        result += chr(index + ord('A'))
        n = (n - 1) // 26

    return result[::-1]

def check_late(a,b,c): # to check late joinin by relaxation of c minutes
    #a-joined time
    #b-lect start time
    #c-relaxation in mins.
    a=list(a)
    b=list(b)

    a[1]=a[1]-c          #subtract c minutes from join time
    a[0]=a[0]+(a[1]//60)
    a[1]=a[1]%60

    if (a[0]<b[0]):
        return 0
    elif (a[0]==b[0]):
        if (a[1]<b[1]):
            return 0
        elif (a[1]==b[1]):
            if (a[2]<=b[2]):
                return 0
            else:
                return 1
        else:
            return 1
    else:
        return 1


def analysis(s_data,l_start,l_end,late_limit,present_limit):
    J = str('Joined')
    L = str('Left')
    Jb = str('Joined before')
    l=len(s_data)
    dis=0
    dur=0
    late=0

    t_j=time(s_data[0])
    t_j1=time_join(t_j , l_start)
    t_l=l_end

    #check late join
    if (check_late(t_j1, l_start, late_limit)==0):
        late = ' '
    else:
        late = 'Late'

    dur=0
    ana_data=[s_data[0][0],t_j1,t_l,late,dis,dur]

    if (l==1):
        if (s_data[0][1]==J or s_data[0][1]==Jb):
            t_j=time(s_data[0])
            t_j=time_join(t_j , l_start)
            t_l=l_end
            pr_time=p_time(t_j , t_l)
            dur=pr_time
        else:
            print('data missing')


    for i in range(1,l):
        if ((s_data[i-1][1]==J or s_data[i-1][1]==Jb) and s_data[i][1]==L):    #check join and left

            t_j=time(s_data[i-1])
            t_j=time_join(t_j , l_start)
            t_l=time(s_data[i])
            t_l=time_left(t_l , l_end)
            pr_time=p_time(t_j , t_l)    #present time
            dur = dur + pr_time
            dis=dis+1


    if ((s_data[-1][1]==J or s_data[-1][1]==Jb) and l != 1):    #check join and left

        t_j=time(s_data[-1])
        t_j=time_join(t_j , l_start)
        t_l=l_end
        pr_time=p_time(t_j , t_l)
        dur = dur + pr_time
        dis=dis+1

    if (t_j1[0] != 12):
        jtime=str(t_j1[0]%12) + str(':') + str(t_j1[1]) + str(':') +str (t_j1[2])
    else :
        jtime=str(t_j1[0]) + str(':') + str(t_j1[1]) + str(':') +str (t_j1[2])

    if (t_l[0] != 12):
        ltime=str(t_l[0]%12) + str(':') + str(t_l[1]) + str(':') +str (t_l[2])
    else:
        ltime=str(t_l[0]) + str(':') + str(t_l[1]) + str(':') +str (t_l[2])

    dur_min=dur//60
    dur_sec=dur%60
    dur_c=str(dur_min)+':'+str(dur_sec)
    if (dur_min < present_limit):
        late = 'PP'

    ana_data=[s_data[0][0],jtime,ltime,late,dis,dur_c]

    return ana_data

#convert time into tuple(hour,minutes,seconds)
def time(std):
    d_t = std[2]
    mode=d_t[-2:]
    if(mode != 'AM' and mode != 'PM'):
        t=d_t[d_t.index(',')+2:]
        th=t[:t.index(':')]
        th=int(th)

    else:
        t=d_t[d_t.index(',')+2:-2]
        th=t[:t.index(':')]
        th=int(th)
        if (mode == 'PM' and th != 12):
            th=th+12

    tm1=t[t.index(':')+1:]
    tm=tm1[:tm1.index(':'):]
    ts=tm1[tm1.index(':')+1:]

    return((int(th),int(tm),int(ts)))

#access time from csv file
def time1(std):
    t = std
    try:
        mode=str(t[-2:])
        if (mode == 'am' or mode=='AM' or mode=='pm' or mode=='PM'):
            pass
        else:
            print('please enter am or pm')
    except:
        print('please enter am or pm')

    try:
        t = t[:-2]
    except:
        t=t

    try :
        th=t[0:t.index('.')]
    except :
        try:
            th=t[0:t.index(':')]
        except:
            th=t

    th = int (th)
    if (mode == 'PM' or mode == 'pm'):
        if (th!=12):
            th=th+12
    #print(th)
    try:
        tm=t[t.index('.')+1:]

    except:
        try:
            tm=t[t.index(':')+1:]

        except:
            tm=0

    return((int(th),int(tm),0))

#present time
def p_time(T1,T2):
    hr=(T2[0]-T1[0])%12
    min=T2[1]-T1[1]
    sec=T2[2]-T1[2]
    t = hr*60*60 + min*60 + sec
    return t

#if joined before lecture then assign join time as lecture start time
def time_join(a , b):
    if (a[0]<b[0]):
        return b
    elif (a[0]>b[0]):
        return a
    else:
        if (a[1]<b[1]):
            return b
        elif (a[1]>b[1]):
            return a
        else:
            if (a[2]<b[2]):
                return b
            elif (a[2]>b[2]):
                return a
            else:
                return a

#if left time is after lecture end time then assign left time as lecture end time
def time_left(a , b):
    if (a[0] < b[0]):
        return a
    elif (a[0] > b[0]):
        return b
    else:
        if (a[1]<b[1]):
            return a
        elif (a[1]>b[1]):
            return b
        else:
            if (a[2]<b[2]):
                return a
            elif (a[2]>b[2]):
                return b
            else:
                return b


#Upload Master Attendance file
check=1
while (check==1):
    try:
        Recordfile=files.upload()

        for key2 in Recordfile:
            pass

        temp_file_name=key2[0:-5]
        i=1
        while (i < 10):
            file_no=10-i
            file_name=temp_file_name+' ('+str(file_no)+').xlsx'
            try:
                wb = load_workbook(file_name)
                break
            except:
                pass
            i=i+1

        if (i==10):
            file_name = temp_file_name +'.xlsx'
            wb = load_workbook(file_name)

        ws=wb.active
        check=0
    except:
        print('Please Upload the Master Attendance file in .xlsx format')
        check=1

lect_col=4
lect_cell = ws.cell(row=5,column=lect_col)
while (lect_cell.value != None):
    lect_col = lect_col+1
    lect_cell = ws.cell(row=5,column=lect_col)

lect_col=lect_col-3

In [ ]:
#@title **STEP2 :** Add or Edit the Attendance { form-width: "20%" }

from openpyxl.styles import PatternFill
from openpyxl.styles import Font, Fill#Connect styles for text
from openpyxl.styles import colors
from openpyxl.styles import Alignment

Choose = "Mark_All_Present" #@param ["Mark_All_Present", "Mark_All_Absent", "Upload_.csv_file", "Enter_Present_nos_manually", "Enter_Absent_nos_manually"]

#Mark_All_Present = True #@param {type:"boolean"}

final_data = []
attd_data = []
date=''
MST_file=''

lect_no = ''
yes_no=0
lect_col=4
lect_cell = ws.cell(row=5,column=lect_col)
while (lect_cell.value != None):
    lect_col = lect_col+1
    lect_cell = ws.cell(row=5,column=lect_col)

lect_col=lect_col-3

print('Attendance of',lect_col-1,'lectures is entered till now.')
check=1
while (check == 1):
    try:
        lect_no = int (input ('Enter lecture number to mark or update the Attendance : '))
        yes_no=0
        if (lect_no==0):
            print('Lecture number 0 is not allowed !, Please enter correct number')
            check=1
        elif (lect_no > lect_col):
            print('Attendance of',lect_col-1,'lectures is entered till now, please enter correct lecture number.')
            check=1
        elif (lect_no <= (lect_col-1) ):
            print()
            print('Attendance for lecture number',lect_no,'is already entered, do you want to edit it?')

            check_edit=1
            edited=0
            while (check_edit==1):
                try:
                    yes_no = int (input('Enter 1 for YES or 0 for NO : '))
                    if (yes_no==1):
                        check=0
                        check_edit=0
                    elif (yes_no==0):
                        check=1
                        check_edit=0
                    else:
                        print('Please enter eiter 1 or 0.')
                        check_edit=1
                except:
                    check_edit=1
        else:
            check=0
    except:
        print('Please Enter Natural Number')
        check=0


# Mark Attendance:

if(Choose != "Upload_.csv_file" and lect_no != ''):
    while(date==''):
        print()
        print('Enter Date: DD/MM/YYYY')

        try:
            dd,mm,yyyy=[int(i) for i in input().split('/')]

            if(yyyy > 2000):
                if(mm > 0 and mm <= 12):
                    if(mm==1 or mm==3 or mm==5 or mm==7 or mm==8 or mm==10 or mm==12):
                        if(dd>0 and dd<= 31):
                            date=str(dd)+'/'+str(mm)+'/'+str(yyyy)

                        else:
                            print('Please enter correct date (dd) !')
                            date=''

                    elif(mm==4 or mm==6 or mm==9 or mm==11):
                        if(dd>0 and dd<=30):
                            date=str(dd)+'/'+str(mm)+'/'+str(yyyy)

                        else:
                            print('Please enter correct date (dd) !')
                            date=''

                    elif(mm==2):
                        if(dd>0 and dd<=29):
                            date=str(dd)+'/'+str(mm)+'/'+str(yyyy)

                        else:
                            print('Please enter correct date (dd) !')
                            date=''

                    else:
                        print('Please enter correct date in DD/MM/YYYY format')
                        date=''

                else:
                    print('Please enter correct month !')
                    date=''
            else:
                print('Please enter correct year !')
                date=''

        except:
            print('Please enter correct date in DD/MM/YYYY format')
            date=''

if(Choose == "Mark_All_Present" and date != ''):

    MST_file =[]    # required from uploaded file
    check==0        # required from uploaded file

    first_roll_no = ws.cell(row=7,column=1).value

    count_students = 7
    while(ws.cell(row=count_students,column=1).value != None):
        count_students = count_students + 1

    last_roll_no = ws.cell(row=count_students-1,column=1).value

    for i in range(first_roll_no , last_roll_no+1):
        final_data.append([i,'','','',0,'60:0'])
        attd_data.append([i,'P'])

elif(Choose == "Mark_All_Absent" and date != ''):

    MST_file =[]    # required from uploaded file
    check==0        # required from uploaded file

    first_roll_no = ws.cell(row=7,column=1).value

    count_students = 7
    while(ws.cell(row=count_students,column=1).value != None):
        count_students = count_students + 1

    last_roll_no = ws.cell(row=count_students-1,column=1).value

    for i in range(first_roll_no , last_roll_no+1):
        attd_data.append([i,'A'])

elif(Choose == "Enter_Present_nos_manually" and date != ''):
    print('Enter Roll Numbers of Students to be marked Present')

    try:
        Present_nos=[int(i) for i in input().split(',')]
        Present_nos.sort()
    except:
        print('Please enter correct information !')

    if(len(Present_nos) != 0):
        MST_file =[]    # required from uploaded file
        check==0        # required from uploaded file

        first_roll_no = ws.cell(row=7,column=1).value

        count_students = 7
        while(ws.cell(row=count_students,column=1).value != None):
            count_students = count_students + 1

        last_roll_no = ws.cell(row=count_students-1,column=1).value

        for j in range(len(Present_nos)):
            for i in range(first_roll_no , last_roll_no+1):
                if(len(str(Present_nos[j]))==1 and str(Present_nos[j])==str(i)[-1:]):
                    final_data.append([i,'','','',0,'60:0'])
                    attd_data.append([i,'P'])
                    break
                elif(len(str(Present_nos[j]))==2 and str(Present_nos[j])==str(i)[-2:]):
                    final_data.append([i,'','','',0,'60:0'])
                    attd_data.append([i,'P'])
                    break
                elif(str(Present_nos[j])==str(i)):
                    final_data.append([i,'','','',0,'60:0'])
                    attd_data.append([i,'P'])
                    break


elif(Choose == "Enter_Absent_nos_manually" and date != ''):
    print('Enter Roll Numbers of Students to be marked Absent')

    try:
        Absent_nos=[int(i) for i in input().split(',')]
        Absent_nos.sort()
    except:
        print('Please enter correct information !')
        Absent_nos=[0]

    MST_file =[]    # required from uploaded file
    check==0        # required from uploaded file

    first_roll_no = ws.cell(row=7,column=1).value

    count_students = 7
    while(ws.cell(row=count_students,column=1).value != None):
        count_students = count_students + 1

    last_roll_no = ws.cell(row=count_students-1,column=1).value

    for i in range(first_roll_no , last_roll_no+1):
        temp = 'P'
        for j in range(len(Absent_nos)):
            if(str(Absent_nos[j])==str(i)):
                temp='A'
                break
            elif(len(str(Absent_nos[j]))==2 and str(Absent_nos[j])==str(i)[-2:]):
                temp='A'
                break
            elif(len(str(Absent_nos[j]))==1 and str(0)+str(Absent_nos[j])==str(i)[-2:]):
                temp='A'
                break
        if(temp=='P'):
            final_data.append([i,'','','',0,'60:0'])
            attd_data.append([i,temp])


elif(Choose == "Upload_.csv_file" and lect_no != ''):
    #Upload excel csv file
    MST_file=[]
    check=1
    version=0
    if (True):
        try:
            print()
            print('Upload Attendance File Downloaded from Microsoft Teams')
            MST_file=files.upload()
            for key in MST_file:
                pass

            unsort_data=[]

            msttemp_file_name=key[0:-4]
            i=1  # To check renamed latest uploaded file
            while (i < 10):
                mstfile_no=10-i
                mstfile_name=msttemp_file_name+' ('+str(mstfile_no)+').csv'
                try:

                    try:
                        df = pd.read_csv(mstfile_name,'rb',delimiter="\t",encoding = 'utf-16')

                    except:
                        col_names = ["Full Name", "Join Time", "Leave Time"]
                        df = pd.read_csv(mstfile_name, names=col_names,delimiter="\t",encoding = 'utf-16')

                    for row in df.iterrows():
                        if(row[1][1] == 'Joined' or row[1][1] == 'Left' or row[1][1] == 'Joined Before'):
                            version = 1
                            break

                        elif(row[1][0] == 'Meeting Summary' or row[1][0] != 'Full Name'):
                            version = 2
                            break

                    if(version == 1):
                        for row in df.iterrows():
                            try:
                                unsort_data.append([int(row[1][0][0:5]),row[1][1],row[1][2]])
                            except:
                                pass

                    elif(version == 2):
                        check_full_name=1
                        for row in df.iterrows():
                            if(row[1][0] == 'Full Name'):
                                break
                            else:
                                check_full_name=check_full_name+1

                        count=0
                        for row in df.iterrows():
                            if(count >= check_full_name):
                                try:
                                    unsort_data.append([int(row[1][0][0:5]),'Joined',row[1][1]])
                                    unsort_data.append([int(row[1][0][0:5]),'Left',row[1][2]])
                                except:
                                    pass
                            count=count+1
                    break

                except:
                    pass

                i = i+1

            if(i==10):
                mstfile_name=msttemp_file_name+'.csv'
                try:
                    try:
                        df = pd.read_csv(mstfile_name,'rb',delimiter="\t",encoding = 'utf-16')

                    except:
                        col_names = ["Full Name", "Join Time", "Leave Time"]
                        df = pd.read_csv(mstfile_name, names=col_names,delimiter="\t",encoding = 'utf-16')

                    for row in df.iterrows():
                        if(row[1][1] == 'Joined' or row[1][1] == 'Left' or row[1][1] == 'Joined Before'):
                            version = 1
                            break

                        elif(row[1][0] == 'Meeting Summary' or row[1][0] != 'Full Name'):
                            version = 2
                            break

                    if(version == 1):
                        for row in df.iterrows():
                            try:
                                unsort_data.append([int(row[1][0][0:5]),row[1][1],row[1][2]])
                            except:
                                pass


                    elif(version == 2):
                        check_full_name=1
                        for row in df.iterrows():
                            if(row[1][0] == 'Full Name'):
                                break
                            else:
                                check_full_name=check_full_name+1

                        count=0
                        for row in df.iterrows():
                            if(count >= check_full_name):
                                try:
                                    unsort_data.append([int(row[1][0][0:5]),'Joined',row[1][1]])
                                    unsort_data.append([int(row[1][0][0:5]),'Left',row[1][2]])
                                except:
                                    pass
                            count=count+1

                except:
                    pass

            if(version != 0):
                unsort_data.sort()
                a_data1=[] # to remove consecutive joins
                a_data_l=[] # to remove consecutive lefts
                a_data=[]
                a_data=unsort_data.copy()

                a_data1=a_data.copy() # to remove consecutive joins
                a_data_l=a_data.copy() # to remove consecutive lefts

                J = str('Joined')
                L = str('Left')
                Jb = str('Joined before')
                check=0
            else:
                print('Wrong file is uploaded !!!')
                check=1

            if (len(unsort_data)==0):
                print('No Data !, Wrong file is uploaded !!!')
                check=1

        except Exception as e:
            print(e,'. Wrong file is uploaded !!!')
            check=1

    print()

    if (len(MST_file) != 0 and check==0):
        check_time=1
        while (check_time==1):
            try:
                print('Lecture Start time (eg 11.15am):')
                start = input()
                start_ext=start[-2:]
                print('Lecture End time (eg 1pm):')
                end = input()
                end_ext=end[-2:]

                if ((start_ext =='am' or start_ext=='AM') and (end_ext=='am' or end_ext=='AM')):
                    lect_start = time1(start)
                    lect_end = time1(end)
                    check_time=0

                elif ((start_ext =='pm' or start_ext=='PM') and (end_ext=='pm' or end_ext=='PM')):
                    lect_start = time1(start)
                    lect_end = time1(end)
                    check_time=0
                elif ((start_ext =='am' or start_ext=='AM') and (end_ext=='pm' or end_ext=='PM')):
                    lect_start = time1(start)
                    lect_end = time1(end)
                    check_time=0

                elif ((start_ext =='pm' or start_ext=='PM') and (end_ext=='am' or end_ext=='AM')):
                    print('make sure that start time is less than end time')
                    check_time=1

                else:
                    print('Please enter correct extension')
                    print('.am or :am or .AM or :AM or .pm or :pm or .PM or :PM')
                    check_time=1
            except:
                check_time=1

        total_lect_time = p_time(lect_start , lect_end)/60

        check_time=1
        while (check_time==1):
            try:
                late_limit = int (5)
                check_time=0
            except:
                check_time=1

        check_time=1
        while (check_time==1):
            try:
                present_limit = (total_lect_time*60)//100
                check_time=0
            except:
                print('Please enter integer only !')
                check_time=1

        count=0
        a2_data=a_data.copy() # to add data in s_data
        s_data=[]
        final_data=[]
        k=0
        while (len(a2_data) != 0):
            if (len(a2_data) != 0):
                s_data.append(a2_data[0])
                a2_data.remove(a2_data[0])
                try:
                    while (s_data[0][0] == a2_data[0][0]):   # check same roll number
                        s_data.append(a2_data[0])
                        a2_data.remove(a2_data[0])

                except:
                    pass

            #print(s_data)
            sorted_data=[s_data[0]]
            for i in range(len(s_data)-1):
                for j in range(len(s_data)-i-1):
                    t0 = time(s_data[j])
                    t1 = time(s_data[j+1])
                    temp = s_data[j].copy()
                    if (t0[0] > t1[0]):
                        s_data[j] = s_data[j+1].copy()
                        s_data[j+1] = temp
                    elif (t0[0] == t1[0]):
                        if (t0[1] > t1[1]):
                            s_data[j] = s_data[j+1].copy()
                            s_data[j+1] = temp
                        elif (t0[1] == t1[1]):
                            if (t0[2] > t1[2]):
                                s_data[j] = s_data[j+1].copy()
                                s_data[j+1] = temp

            temp_s_data=s_data.copy()
            len_s_data = len(temp_s_data)
            for i1 in range(len_s_data):
                itom=temp_s_data[i1].copy()
                if (itom[1]==L):
                    t_l = time(itom)
                    if (t_l[0] < lect_start[0]):
                        s_data.remove(itom)
                        try:
                            s_data.remove(temp_s_data[i1-1])
                        except:
                            pass
                    elif (t_l[0] == lect_start[0]):
                        if (t_l[1] < lect_start[1]):
                            s_data.remove(itom)
                            try:
                                s_data.remove(temp_s_data[i1-1])
                            except:
                                pass
                        elif (t_l[1] == lect_start[1]):
                            if (t_l[2] <= lect_start[2]):
                                s_data.remove(itom)
                                try:
                                    s_data.remove(temp_s_data[i1-1])
                                except:
                                    pass

            temp_s_data=s_data.copy()
            for itom in temp_s_data:
                if (itom[1]==L):
                    t_l = time(itom)
                    if (t_l[0] > lect_end[0]):
                        s_data.remove(itom)
                    elif (t_l[0] == lect_end[0]):
                        if (t_l[1] > lect_end[1]):
                            s_data.remove(itom)
                        elif (t_l[1] == lect_end[1]):
                            if (t_l[2] >= lect_end[2]):
                                s_data.remove(itom)

            temp_s_data=s_data.copy()
            for itom in temp_s_data:
                if (itom[1]==J or Jb):
                    t_j = time(itom)
                    if (t_j[0] > lect_end[0]):
                        s_data.remove(itom)
                    elif (t_j[0] == lect_end[0]):
                        if (t_j[1] > lect_end[1]):
                            s_data.remove(itom)
                        elif (t_j[1] == lect_end[1]):
                            if (t_j[2] >= lect_end[2]):
                                s_data.remove(itom)

            len_s1=len(s_data)
            s1_data=s_data.copy()
            for i in range (1,len_s1): #df2 - sorted data in pandas

                if (s1_data[i-1][0]==s1_data[i][0]):
                    if (s1_data[i-1][1]!=L and s1_data[i][1]!=L): #remove consecutive join of same person
                        s_data.remove(s1_data[i])

            s2_data=s_data.copy()
            len_s2=len(s_data)
            for j in range(1,len_s2):
                if (s2_data[j-1][0]==s2_data[j][0]):
                    if (s2_data[j-1][1]==L and s2_data[j][1]==L): #remove consecutive left of same person
                        s_data.remove(s2_data[j-1])
            try:
                final_data.append(analysis(s_data,lect_start,lect_end,late_limit,present_limit))
            except:
                pass
            s_data=[]
        #print(final_data)
        attd_data=[]    #Attendance Roll No. wise
        for i in range(len(final_data)):
            try:
                roll_no = int (final_data[i][0])

                if (isinstance(roll_no , int)):
                    if (final_data[i][3]=='Late'):
                        attd = 'A'
                    elif (final_data[i][3]=='PP'):
                        attd = 'PP'
                    else:
                        attd = 'P'
                attd_data.append([roll_no , attd])

            except:
                pass

        # Partially Present Students
        print()
        print('Following Students are Partially Present')
        count_prints=0
        for student in final_data:
            if(student[3] == 'PP'):
                if(count_prints < 8):
                    print(student[0],'(',str(student[5]),')',sep='',end=' ; ')

                else:
                    print()
                    print(student[0],'(',str(student[5]),')',sep='',end=' ; ')
                    count_prints=0

                count_prints=count_prints+1

        # following partially present students are to be mark present
        print()
        print()
        print('Enter Roll Numbers of Students to be marked Present')

        try:
            pp_to_p=[int(i) for i in input().split(',')]
            pp_to_p.sort()
        except:
            pp_to_p=[0]

        mark_present=[]
        division_ext=str(final_data[0][0])[:3]
        for i in pp_to_p:
            if(len(str(i))==1):
                mark_present.append(int(division_ext+str(0)+str(i)))
            elif(len(str(i))==2):
                mark_present.append(int(division_ext+str(i)))
            elif(len(str(i))==5):
                mark_present.append(i)

        for student1 in mark_present:
            check_student=0
            for student2 in final_data:
                if(student1 == student2[0]):
                    check_student=1
                    final_data.remove(student2)
                    final_data.append([student1,'','','',0,str(int(total_lect_time))+':00'])
                    break
            if(check_student==0):
                final_data.append([student1,'','','',0,str(int(total_lect_time))+':00'])

        final_data.sort()

        for student1 in mark_present:
            check_student=0
            for student2 in attd_data:
                if(student1 == student2[0]):
                    check_student=1
                    attd_data.remove(student2)
                    attd_data.append([student1,'P'])
                    break
            if(check_student==0):
                attd_data.append([student1,'P'])

        attd_data.sort()


# Fill Attendance in excel file
if ((Choose == "Upload_.csv_file" and len(MST_file) != 0 and check==0 and lect_no != '') or (Choose != "Upload_.csv_file" and date != '')):
    check_cat=1
    absent=[]
    partially_present=[]
    try:
        row0 = 7
        col0 = lect_no + 3
        wcell = ws.cell(5,col0)
        wcell.value = lect_no
        wcell.alignment = Alignment(horizontal='center',vertical='center')


        if(Choose == "Upload_.csv_file"):
            date=a_data[1][2]
            date=date[:date.index(',')]

        wcell = ws.cell(6,col0)
        wcell.value = date
        wcell.alignment = Alignment(horizontal='center',vertical='center')
        cell_obj=ws.cell(row = row0 ,column = 1)
        i = 0
        while (cell_obj.value != None):
            cell_obj=ws.cell(row = row0 ,column = 1)
            check_exist=1
            if ( isinstance(cell_obj.value , int)):
                try:
                    if (int(cell_obj.value) == int (attd_data[i][0])):
                        wcell=ws.cell(row0,col0)
                        wcell.style='Normal'
                        if (final_data[i][3] == 'PP'and final_data[i][4] < 5):  #if discontinued more no. of then times mark present
                            wcell.value='PP'
                            absent.append(str(cell_obj.value)+ ' ('+str(final_data[i][5])+')')
                            wcell.alignment = Alignment(horizontal='center',vertical='center')
                            wcell.fill=PatternFill(start_color="feedc6", end_color="feedc6", fill_type="solid")
                            #wcell.font = Font(color = colors.RED)
                        else:
                            if (final_data[i][3] == 'Late'):
                                wcell.fill=PatternFill(start_color="fcf9a6", end_color="feedc6", fill_type="solid")
                                #wcell.font = Font(color = colors.RED)

                            check=1
                            k=1
                            m=0
                            while (check==1 and k < lect_no ):
                                pr_cell=ws.cell(row0,col0 - k)
                                if (isinstance(pr_cell.value , int)):
                                    m = pr_cell.value
                                    check=0
                                else:
                                    k=k+1
                                    check=1
                            wcell.value=m+1  # mark attendance count
                            wcell.alignment = Alignment(horizontal='center',vertical='center')

                            if (k == lect_no):
                                wcell.value=1
                                wcell.alignment = Alignment(horizontal='center',vertical='center')

                        i=i+1
                        check_exist=1

                    else:
                        if (cell_obj.value < int (attd_data[i][0])):
                            wcell=ws.cell(row0,col0)
                            wcell.style='Normal'
                            wcell.value='A'
                            absent.append(str(cell_obj.value))
                            wcell.alignment = Alignment(horizontal='center',vertical='center')
                            wcell.fill=PatternFill(start_color="feedc6", end_color="feedc6", fill_type="solid")
                            #wcell.font = Font(color = colors.RED)
                            #row0=row0+1
                            check_exist=1
                        else:
                            i=i+1
                            check_exist=0
                except:
                    wcell=ws.cell(row0,col0)
                    wcell.style='Normal'
                    wcell.value='A'
                    absent.append(str(cell_obj.value))
                    wcell.alignment = Alignment(horizontal='center',vertical='center')
                    wcell.fill=PatternFill(start_color="feedc6", end_color="feedc6", fill_type="solid")
                    #wcell.font = Font(color = colors.RED)

            if (check_exist==1):
                col_edit0 = col0
                cell_edit0 = ws.cell(row0 , col_edit0)
                m1 = cell_edit0.value

                while (col_edit0 > 3 and isinstance(m1 , int) == 0):
                    col_edit0 = col_edit0 - 1
                    cell_edit0 = ws.cell(row0 , col_edit0)
                    m1 = cell_edit0.value
                if (isinstance(m1 , int) == 0):
                    m1=0

                col_edit1 = col0+1
                cell_edit1 = ws.cell(row0 , col_edit1)
                check_cell=ws.cell(row0,col_edit1+3)

                while (check_cell.value != None):

                    if (isinstance(cell_edit1.value , int)==1):
                        cell_edit1.value = m1+1
                        col_edit1 = col_edit1+1
                        cell_edit1 = ws.cell(row0 , col_edit1)
                        check_cell=ws.cell(row0,col_edit1+3)
                        m1=m1+1

                    else:
                        col_edit1 = col_edit1+1
                        cell_edit1 = ws.cell(row0 , col_edit1)
                        check_cell=ws.cell(row0,col_edit1+3)

                if (check_cat==1):
                    att_cell = ws.cell(row0-1 , col_edit1)
                    con_cell = ws.cell(row0-1 , col_edit1+1)
                    per_cell = ws.cell(row0-1 , col_edit1+2)

                    att_cell.value='Att'
                    att_cell.alignment = Alignment(horizontal='center',vertical='center')
                    con_cell.value='Con'
                    con_cell.alignment = Alignment(horizontal='center',vertical='center')
                    per_cell.value='Per(%)'
                    per_cell.alignment = Alignment(horizontal='center',vertical='center')

                    check_cat=0

                att_cell = ws.cell(row0 , col_edit1)
                att_cell.alignment = Alignment(horizontal='center',vertical='center')
                con_cell = ws.cell(row0 , col_edit1+1)
                con_cell.alignment = Alignment(horizontal='center',vertical='center')
                per_cell = ws.cell(row0 , col_edit1+2)
                per_cell.alignment = Alignment(horizontal='center',vertical='center')
                att_cell.value="=COUNT(D{}:{}{})".format(row0,getColumnName(col_edit1-1),row0)
                att_cell.alignment = Alignment(horizontal='center',vertical='center')
                con_cell.value=col_edit1-4
                con_cell.alignment = Alignment(horizontal='center',vertical='center')
                l_con=col_edit1-4

                per_cell.value="={}{}*100/{}{}".format(getColumnName(col_edit1),row0,getColumnName(col_edit1+1),row0)
                per_cell.alignment = Alignment(horizontal='center',vertical='center')

                row0 = row0+1
                cell_obj=ws.cell(row = row0 ,column = 1)

    except:
        pass

    lect_col=lect_col+1
    try:
        if (yes_no==1):
            lect_col=lect_col-1
        check_edit=1
    except:
        pass

    print('----------------------------------------------------------------------------------------')
    expr = 'Absent (total = ' + str(len(absent)) + ')'
    print(colored(expr,'blue', attrs=['bold']))

    j=0
    for i in range(len(absent)):
        if (j<8):
            if (len(absent[i]) > 5):
                print(colored(absent[i],'blue', attrs=['bold']),end=' ; ')
            else:
                print(colored(absent[i], attrs=['bold']),end=' ; ')
        else:
            print()
            if (len(absent[i]) > 5):
                print(colored(absent[i],'blue', attrs=['bold']),end=' ; ')
            else:
                print(colored(absent[i], attrs=['bold']),end=' ; ')

            j = 0
        j=j+1

Choose == "Upload_.csv_file"
date=''

Repeat **STEP2** to add attendance for multiple lectures

In [ ]:
#@title **STEP3 :** Download Master File
down_file = temp_file_name+'(1-'+str(lect_col-1)+').xlsx'

wb.save(down_file)
files.download(down_file)